# Docker Basics

Docker is a platform that allows you to automate the deployment of applications inside lightweight, portable containers. This tutorial will guide you through the basics of Docker: installation, pulling and running containers, building Docker images, and basic container management.

Before we begin, let's check if Docker is installed and running on your machine.

## Checking Docker Installation

In [ ]:
!docker --version

If Docker is installed, this will display the installed version. If not, please install Docker before proceeding. To avoid complex setup, it is recommended to install [Docker Desktop](https://docs.docker.com/desktop/).

## Pulling Your First Docker Image
Docker containers are built from Docker images. In this section, we'll pull a pre-built image from [Docker Hub](https://hub.docker.com/).

We'll start by pulling the official `hello-world` image.

In [ ]:
!docker pull hello-world

On Linux: If you received a 'permission denied' error, try following the instructions at: https://www.digitalocean.com/community/questions/how-to-fix-docker-got-permission-denied-while-trying-to-connect-to-the-docker-daemon-socket in a terminal window.

## Running a Docker Container
Now that we have the `hello-world` image, let's run it as a container.

In [ ]:
!docker run hello-world

When you run this, Docker will start a container using the `hello-world` image. It will print a message and then exit. The message demonstrates that your Docker installation is working properly.

## Listing Running and Stopped Containers
Docker keeps a history of the containers you've run. You can list them using the following command:

In [ ]:
!docker ps -a

This command will display all containers, including stopped ones, along with their ID, name, status, and other details.

## Removing a Docker Container
To clean up, let's remove the `hello-world` container we just created.

First, find the container ID or name using `docker ps -a`, and then run, replace the ID in the next command and run:

In [ ]:
!docker rm 1faa6223cdf4

Let's verify that the container was removed.

In [ ]:
!docker ps -a

## Using Shared Volumes in Docker

Shared volumes allow you to share files between your host machine and the Docker container. This is useful for editing files on your local machine while having those changes immediately reflected in the container.

We'll create a shared volume between a folder on your host machine and the container running a simple Python script.

###
First, let's create a directory on your host machine where we can store a Python script.

In [ ]:
# Create directory for the shared volume in python
import os
print("Creating directory in folder: " + os.getcwd())
os.makedirs("shared_volume", exist_ok=True)

### Create a Python Script on Your Host

In [ ]:
%%writefile ./shared_volume/hello_from_volume.py
print("Hello from the shared volume!")

## Building a Docker Image
Now, let's move on to creating our own Docker image. We'll create a simple image that runs a Python script from the shared volume.

First, create a `Dockerfile`. This file contains instructions for building a Docker image. Here's an example:

In [ ]:
%%writefile Dockerfile
# Use Python base image
FROM python:3.8-slim

# Set the working directory inside the container
WORKDIR /app

# Command to run a script from the volume (this script will be provided by the volume)
CMD ["python", "/app/hello_from_volume.py"]

Now, build the Docker image using the `docker build` command. The `-t` flag allows you to tag the image with a name:

In [ ]:
!docker build -t my-python-app .

## Running the Custom Docker Image

We will now run the container, mounting the local directory (`./shared_volume`) into the container at the `/app` directory. This way, the Python script on your local machine will be accessible from inside the container.

Explanation:
- **`-v "{shared_vol_dir}:/app"`**: This option mounts the host directory (`{shared_vol_dir}`) to the container’s `/app` directory.
- **`--rm`**: Automatically removes the container once it exits.
- **`my-python-app`**: This is the image we built earlier.

In [ ]:
shared_vol_dir = os.path.abspath("./shared_volume")
shared_vol_dir

In [ ]:
!docker run --rm -v "{shared_vol_dir}:/app" my-python-app

You should see the output: `Hello from the shared volume!`.

## Editing the Script

Let’s modify the Python script from your local machine and re-run the container to see how changes are reflected:

In [ ]:
%%writefile ./shared_volume/hello_from_volume.py
print("Hello again from the updated shared volume!")

Run the container again:

In [ ]:
!docker run --rm -v "{shared_vol_dir}:/app" my-python-app

The container will now output: `Hello again from the updated shared volume!`.


## Clean Up

To remove the image you created:

In [ ]:
!docker rmi my-python-app

In [ ]:
# Remove all stopped containers
!docker container prune -f